# 导入工具包

In [1]:
import random
import numpy as np
import pandas as pd
from bert4keras.backend import keras
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense

Using TensorFlow backend.


In [2]:
!pip install tensorflow-gpu==2.2.0 -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 查看新下载模型

In [3]:
!ls ../model/

albert_small_zh_google	chinese_wwm_ext_L-12_H-768_A-12


In [4]:
!ls ../model/chinese_wwm_ext_L-12_H-768_A-12

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [2]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 查看框架支持的模型

In [3]:
build_transformer_model?

In [4]:
!cat ~/opt/anaconda3/lib/python3.7/site-packages/bert4keras/models.py

cat: /root/opt/anaconda3/lib/python3.7/site-packages/bert4keras/models.py: 没有那个文件或目录


# 加载预训练模型

In [5]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='roberta',
    return_keras_model=False,
)

In [6]:
output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [7]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  # 用足够小的学习率
    metrics=['accuracy'],
)

# 生成数据

In [8]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    # df_train_data = pd.read_csv("../data/train.csv")
    # df_test_data = pd.read_csv("../data/test.csv")
    
    df_train_data = pd.read_csv("../data/train.csv").drop_duplicates(['level_1', 'level_2', 'level_3', 'level_4', 'content','label'])
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [9]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

In [10]:
train_data

[(1, '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t3、消防设施、器材和消防安全标志是否在位、完整；\t一般', 1),
 (3,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t3、消防设施、器材和消防安全标志是否在位、完整；\t消防通道有货物摆放 清理不及时',
  0),
 (4,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用；\t防火门打开状态',
  0),
 (5,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t8、易燃易爆危险物品和场所防火防爆措施的落实情况以及其他重要物资的防火安全情况；\t防爆柜里面稀释剂，机油费混装',
  0),
 (6,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t已经整改',
  1),
 (7,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t逃生通道有货物阻挡。',
  0),
 (11,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t通道有堆放货物',
  0),
 (12, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t10、防火巡查情况；\t灭火器没检查，已检查，已整改', 0),
 (13,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t1、消防安全重点单位应当进行每日防火巡查，并确定巡查的人员、内容、部位和频次。其他单位可以根据需要组织防火巡查。巡查的内容应当包括：1、用火、用电有无违章情况；\t三楼车间天花板脱落，找人维修',
  0),
 (15,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t有一处安全出口不畅通',
  0),
 (16,
  '商

In [11]:
# valid_data

In [12]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [13]:
# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

# 验证和预测

In [14]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

In [15]:
class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

In [16]:
def data_pred(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = int(model.predict([token_ids, segment_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

# 训练和验证

In [18]:
evaluator = Evaluator()

In [19]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=15,
        callbacks=[evaluator]
    )

/root/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/15
233/233 [==============================] - 117s 504ms/step - loss: 0.1596 - accuracy: 0.9478
val_acc: 0.97131, best_val_acc: 0.97131, test_acc: 0.97131

Epoch 2/15
233/233 [==============================] - 107s 459ms/step - loss: 0.0618 - accuracy: 0.9789
val_acc: 0.97699, best_val_acc: 0.97699, test_acc: 0.97699

Epoch 3/15
233/233 [==============================] - 107s 459ms/step - loss: 0.0414 - accuracy: 0.9862
val_acc: 0.97825, best_val_acc: 0.97825, test_acc: 0.97825

Epoch 4/15
233/233 [==============================] - 107s 459ms/step - loss: 0.0272 - accuracy: 0.9911
val_acc: 0.97982, best_val_acc: 0.97982, test_acc: 0.97982

Epoch 5/15
233/233 [==============================] - 107s 460ms/step - loss: 0.0190 - accuracy: 0.9944
val_acc: 0.97699, best_val_acc: 0.97982, test_acc: 0.97699

Epoch 6/15
233/233 [==============================] - 106s 457ms/step - loss: 0.0185 - accuracy: 0.9940
val_acc: 0.97573, best_val_acc: 0.97982, test_acc: 0.97573

Epoch 7/15
233/2

# 加载训练最好的模型

In [20]:
model.load_weights('best_model.weights')
# print(u'final test acc: %05f\n' % (evaluate(test_generator)))
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.980454



In [21]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.999060



# 模型预测

In [22]:
id_ids, y_pred_ids = data_pred(test_data)

In [23]:
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

In [24]:
df_save.to_csv('result.csv')